<a href="https://colab.research.google.com/github/parekhakhil/pyImageSearch/blob/main/1003_hyperparameter_tuning_tf_scikit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter tuning for Deep Learning with scikit-learn, Keras, and TensorFlow




This notebook is associated with the [Hyperparameter tuning for Deep Learning with scikit-learn, Keras, and TensorFlow](https://www.pyimagesearch.com/2021/05/31/hyperparameter-tuning-for-deep-learning-with-scikit-learn-keras-and-tensorflow/) blog post published on 2021-05-31.

Only the code for the blog post is here. Most codeblocks have a 1:1 relationship with what you find in the blog post with two exceptions: (1) Python classes are not separate files as they are typically organized with PyImageSearch projects, and (2) Command Line Argument parsing is replaced with an `args` dictionary that you can manipulate as needed.

We recommend that you execute (press ▶️) the code block-by-block, as-is, before adjusting parameters and `args` inputs. Once you've verified that the code is working, you are welcome to hack with it and learn from manipulating inputs, settings, and parameters. For more information on using Jupyter and Colab, please refer to these resources:

*   [Jupyter Notebook User Interface](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html#notebook-user-interface)
*   [Overview of Google Colaboratory Features](https://colab.research.google.com/notebooks/basic_features_overview.ipynb)



Happy hacking!



<hr>


### Download the code zip file

In [ ]:
!wget https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/hyperparameter-tuning-tf-scikit/hyperparameter-tuning-tf-scikit.zip
!unzip -qq hyperparameter-tuning-tf-scikit.zip
%cd hyperparameter-tuning-tf-scikit

## Blog Post Code

### Import Packages

In [ ]:
# import tensorflow and fix the random seed for better reproducibility
import tensorflow as tf
tf.random.set_seed(42)

# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.datasets import mnist

### Implementing our basic feedforward neural network

In [ ]:
def get_mlp_model(hiddenLayerOne=784, hiddenLayerTwo=256,
	dropout=0.2, learnRate=0.01):
	# initialize a sequential model and add layer to flatten the
	# input data
	model = Sequential()
	model.add(Flatten())

	# add two stacks of FC => RELU => DROPOUT
	model.add(Dense(hiddenLayerOne, activation="relu",
		input_shape=(784,)))
	model.add(Dropout(dropout))
	model.add(Dense(hiddenLayerTwo, activation="relu"))
	model.add(Dropout(dropout))

	# add a softmax layer on top
	model.add(Dense(10, activation="softmax"))

	# compile the model
	model.compile(
		optimizer=Adam(learning_rate=learnRate),
		loss="sparse_categorical_crossentropy",
		metrics=["accuracy"])

	# return compiled model
	return model

### Creating our basic training script (no hyperparameter tuning)

In [ ]:
# load the MNIST dataset
print("[INFO] downloading MNIST...")
((trainData, trainLabels), (testData, testLabels)) = mnist.load_data()

# scale data to the range of [0, 1]
trainData = trainData.astype("float32") / 255.0
testData = testData.astype("float32") / 255.0

In [ ]:
# initialize our model with the default hyperparameter values
print("[INFO] initializing model...")
model = get_mlp_model()

# train the network (i.e., no hyperparameter tuning)
print("[INFO] training model...")
H = model.fit(x=trainData, y=trainLabels,
	validation_data=(testData, testLabels),
	batch_size=8,
	epochs=20)

# make predictions on the test set and evaluate it
print("[INFO] evaluating network...")
accuracy = model.evaluate(testData, testLabels)[1]
print("accuracy: {:.2f}%".format(accuracy * 100))

[INFO] initializing model...
[INFO] training model...
Epoch 1/20
7500/7500 [==============================] - 50s 7ms/step - loss: 0.8856 - accuracy: 0.7734 - val_loss: 0.4093 - val_accuracy: 0.9022
Epoch 2/20
7500/7500 [==============================] - 47s 6ms/step - loss: 0.6730 - accuracy: 0.8408 - val_loss: 0.4901 - val_accuracy: 0.8644
Epoch 3/20
7500/7500 [==============================] - 49s 7ms/step - loss: 0.6676 - accuracy: 0.8507 - val_loss: 0.4805 - val_accuracy: 0.9021
Epoch 4/20
7500/7500 [==============================] - 46s 6ms/step - loss: 0.6591 - accuracy: 0.8534 - val_loss: 0.4451 - val_accuracy: 0.8951
Epoch 5/20
7500/7500 [==============================] - 47s 6ms/step - loss: 0.6643 - accuracy: 0.8514 - val_loss: 0.4343 - val_accuracy: 0.8898
Epoch 6/20
7500/7500 [==============================] - 48s 6ms/step - loss: 0.6509 - accuracy: 0.8541 - val_loss: 0.4318 - val_accuracy: 0.8924
Epoch 7/20
7500/7500 [==============================] - 48s 6ms/step - loss:

### Implementing our Keras/TensorFlow hyperparameter tuning script

In [ ]:
# wrap our model into a scikit-learn compatible classifier
print("[INFO] initializing model...")
model = KerasClassifier(build_fn=get_mlp_model, verbose=0)

# define a grid of the hyperparameter search space
hiddenLayerOne = [256, 512, 784]
hiddenLayerTwo = [128, 256, 512]
learnRate = [1e-2, 1e-3, 1e-4]
dropout = [0.3, 0.4, 0.5]
batchSize = [4, 8, 16, 32]
epochs = [10, 20, 30, 40]

# create a dictionary from the hyperparameter grid
grid = dict(
	hiddenLayerOne=hiddenLayerOne,
	learnRate=learnRate,
	hiddenLayerTwo=hiddenLayerTwo,
	dropout=dropout,
	batch_size=batchSize,
	epochs=epochs
)

In [ ]:
# initialize a random search with a 3-fold cross-validation and then
# start the hyperparameter search process
print("[INFO] performing random search...")
searcher = RandomizedSearchCV(estimator=model, n_jobs=-1, cv=3,
	param_distributions=grid, scoring="accuracy")
searchResults = searcher.fit(trainData, trainLabels)

# summarize grid search information
bestScore = searchResults.best_score_
bestParams = searchResults.best_params_
print("[INFO] best score is {:.2f} using {}".format(bestScore,
	bestParams))

In [ ]:
# extract the best model, make predictions on our data, and show a
# classification report
print("[INFO] evaluating the best model...")
bestModel = searchResults.best_estimator_
accuracy = bestModel.score(testData, testLabels)
print("accuracy: {:.2f}%".format(accuracy * 100))

For a detailed walkthrough of the concepts and code, be sure to refer to the full tutorial, [*Hyperparameter tuning for Deep Learning with scikit-learn, Keras, and TensorFlow*](https://www.pyimagesearch.com/2021/05/31/hyperparameter-tuning-for-deep-learning-with-scikit-learn-keras-and-tensorflow/) published on 2021-05-31.